In [26]:
import pandas as pd
from osgeo import gdal
import math
import os
import glob
from PIL import Image # PIL library supports only TIF format but not newer TIFF
import numpy as np
import shutil

In [27]:
data_directory = '../data/'
asf_data_directory = data_directory + '3_processed_ASF_data/'
interpolated_AIS_data_directory = data_directory + '5_interpolated_AIS_data/'
#annotated_asf_data_directory = data_directory + '6_annotated_ASF_data/'
cutted_asf_data_directory = data_directory + '6_cutted_ASF_data/'
final_dataset = data_directory + '7_final_dataset/'
final_dataset_images = final_dataset + 'images/'
final_dataset_labels = final_dataset + 'labels/'
###
second_final_dataset = data_directory + '8_final_dataset_second_version/'
second_test = second_final_dataset + 'test/'
second_train = second_final_dataset + 'train/'
second_val = second_final_dataset + 'val/'
#sliced_asf_data_directory = data_directory +'7_sliced_ASF_data/'
results_directory = '../results/'

### 1. Create train/test split

In [28]:
stopfiles = ['.gitkeep', '.ipynb_checkpoints']
TEST_RATIO = 0.3
VALIDATION_RATIO = 0.3

pictures=list()
for file in os.listdir(cutted_asf_data_directory):
    if not file in stopfiles:
        pictures.append(file)

display(pictures)
total_number_of_pictures = len(pictures)
print(f"total number_of_folder_pictures: {total_number_of_pictures}")
print()
nr_test_pictures = round(TEST_RATIO*total_number_of_pictures)
print(f"number_of_test_folder_pictures: {nr_test_pictures}")
nr_train_pictures = total_number_of_pictures - nr_test_pictures
print(f"number_of_train_folder_pictures: {nr_train_pictures}")
print()
nr_val_pictures = round(VALIDATION_RATIO*nr_train_pictures)
print(f"nr_val_folder_pictures: {nr_val_pictures}")

['S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC',
 'S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC',
 'S1B_IW_GRDH_1SDV_20210306T015735_20210306T015804_025888_031671_7F65_TC']

total number_of_folder_pictures: 3

number_of_test_folder_pictures: 1
number_of_train_folder_pictures: 2

nr_val_folder_pictures: 1


In [29]:
test_folders = pictures[0:nr_test_pictures]
print(f"test_folders: {test_folders}")
print()
train_folders = pictures[nr_test_pictures:]
print(f"train_folders: {train_folders}")
print()
val_folders = pictures[nr_test_pictures:nr_test_pictures+nr_val_pictures]
print(f"val_folders: {val_folders}")

test_folders: ['S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC']

train_folders: ['S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC', 'S1B_IW_GRDH_1SDV_20210306T015735_20210306T015804_025888_031671_7F65_TC']

val_folders: ['S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC']


### 2. Copy from folders only pictures which has corresponding txt file

In [30]:
FORMAT_TO_COPY = 'jpg' # 'tif'

In [31]:
def copy_pictures_and_labels(source_path, source_images_folders, destination_path_images, destination_path_labels, destination_format):
    source_path = source_path
    source_images_folders = source_images_folders
    destination_path_images = destination_path_images
    destination_path_labels = destination_path_labels
    destination_format = destination_format
    print(destination_path_labels)

    for folder in source_images_folders:
        print()
        print(folder)
        # choose only files with txt annotations
        print(os.listdir(source_path + folder))
        print()
        labels=list()
        images=list()
        for file in os.listdir(source_path + folder):
            if file.endswith(".txt"):
                labels.append(file)
                image_file = file.replace(".txt", ".tif")
                images.append(image_file)
        # copy files - rename them while copying
        # labels
        for file in labels:
            src = source_path + '/' + folder + '/' + file
            dst = destination_path_labels + folder + '_' + file
            shutil.copy(src, dst)
        # pictures
        for file in images:
            src = source_path + '/' + folder + '/' + file
            dst = destination_path_images + folder + '_' + file.strip('.tif') + '.' + destination_format
            shutil.copy(src, dst)

2.0 Optionally - delete the contents of the files

In [32]:
files = glob.glob(f'{final_dataset}*/*/*.jpg')
for f in files:
    os.remove(f)
files = glob.glob(f'{final_dataset}*/*/*.tif')
for f in files:
    os.remove(f)
files = glob.glob(f'{final_dataset}*/*/*.txt')
for f in files:
    os.remove(f)

2.1 Test pictures

In [33]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = test_folders,
                        destination_path_images = final_dataset_images + 'test/',
                        destination_path_labels = final_dataset_labels + 'test/',
                        destination_format=FORMAT_TO_COPY)

../data/7_final_dataset/labels/test/

S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC
['subset1.tif', 'subset10.tif', 'subset100.tif', 'subset101.tif', 'subset101.txt', 'subset102.tif', 'subset103.tif', 'subset104.tif', 'subset105.tif', 'subset106.tif', 'subset107.tif', 'subset108.tif', 'subset109.tif', 'subset11.tif', 'subset110.tif', 'subset111.tif', 'subset112.tif', 'subset113.tif', 'subset114.tif', 'subset114.txt', 'subset115.tif', 'subset116.tif', 'subset117.tif', 'subset118.tif', 'subset119.tif', 'subset12.tif', 'subset120.tif', 'subset13.tif', 'subset13.txt', 'subset14.tif', 'subset15.tif', 'subset16.tif', 'subset17.tif', 'subset18.tif', 'subset19.tif', 'subset2.tif', 'subset20.tif', 'subset21.tif', 'subset22.tif', 'subset22.txt', 'subset23.tif', 'subset24.tif', 'subset25.tif', 'subset26.tif', 'subset27.tif', 'subset28.tif', 'subset29.tif', 'subset3.tif', 'subset30.tif', 'subset31.tif', 'subset32.tif', 'subset33.tif', 'subset34.tif', 'subset34.txt', 'subse

2.2 Train pictures

In [34]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = train_folders,
                        destination_path_images = final_dataset_images + 'train/',
                        destination_path_labels = final_dataset_labels + 'train/',
                        destination_format=FORMAT_TO_COPY)

../data/7_final_dataset/labels/train/

S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC
['subset1.tif', 'subset10.tif', 'subset100.tif', 'subset100.txt', 'subset101.tif', 'subset101.txt', 'subset102.tif', 'subset102.txt', 'subset103.tif', 'subset104.tif', 'subset105.tif', 'subset106.tif', 'subset107.tif', 'subset108.tif', 'subset109.tif', 'subset11.tif', 'subset110.tif', 'subset111.tif', 'subset112.tif', 'subset112.txt', 'subset113.tif', 'subset114.tif', 'subset114.txt', 'subset115.tif', 'subset116.tif', 'subset117.tif', 'subset118.tif', 'subset119.tif', 'subset12.tif', 'subset120.tif', 'subset121.tif', 'subset122.tif', 'subset123.tif', 'subset124.tif', 'subset125.tif', 'subset126.tif', 'subset127.tif', 'subset128.tif', 'subset129.tif', 'subset13.tif', 'subset130.tif', 'subset131.tif', 'subset132.tif', 'subset14.tif', 'subset15.tif', 'subset16.tif', 'subset17.tif', 'subset18.tif', 'subset19.tif', 'subset2.tif', 'subset20.tif', 'subset21.tif', 'subset22.tif', 'subs

2.3 Validation pictures

In [35]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = val_folders,
                        destination_path_images = final_dataset_images + 'val/',
                        destination_path_labels = final_dataset_labels + 'val/',
                        destination_format=FORMAT_TO_COPY)

../data/7_final_dataset/labels/val/

S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC
['subset1.tif', 'subset10.tif', 'subset100.tif', 'subset100.txt', 'subset101.tif', 'subset101.txt', 'subset102.tif', 'subset102.txt', 'subset103.tif', 'subset104.tif', 'subset105.tif', 'subset106.tif', 'subset107.tif', 'subset108.tif', 'subset109.tif', 'subset11.tif', 'subset110.tif', 'subset111.tif', 'subset112.tif', 'subset112.txt', 'subset113.tif', 'subset114.tif', 'subset114.txt', 'subset115.tif', 'subset116.tif', 'subset117.tif', 'subset118.tif', 'subset119.tif', 'subset12.tif', 'subset120.tif', 'subset121.tif', 'subset122.tif', 'subset123.tif', 'subset124.tif', 'subset125.tif', 'subset126.tif', 'subset127.tif', 'subset128.tif', 'subset129.tif', 'subset13.tif', 'subset130.tif', 'subset131.tif', 'subset132.tif', 'subset14.tif', 'subset15.tif', 'subset16.tif', 'subset17.tif', 'subset18.tif', 'subset19.tif', 'subset2.tif', 'subset20.tif', 'subset21.tif', 'subset22.tif', 'subset

# 3. Second version of dataset creation

2.0 Optionally - delete the contents of the files

In [36]:
files = glob.glob(f'{second_final_dataset}*/*/*.jpg')
for f in files:
    os.remove(f)
files = glob.glob(f'{second_final_dataset}*/*/*.tif')
for f in files:
    os.remove(f)
files = glob.glob(f'{second_final_dataset}*/*/*.txt')
for f in files:
    os.remove(f)

2.1 Test pictures

In [37]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = test_folders,
                        destination_path_images = second_test + 'images/',
                        destination_path_labels = second_test + 'labels/',
                        destination_format=FORMAT_TO_COPY)

../data/8_final_dataset_second_version/test/labels/

S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC
['subset1.tif', 'subset10.tif', 'subset100.tif', 'subset101.tif', 'subset101.txt', 'subset102.tif', 'subset103.tif', 'subset104.tif', 'subset105.tif', 'subset106.tif', 'subset107.tif', 'subset108.tif', 'subset109.tif', 'subset11.tif', 'subset110.tif', 'subset111.tif', 'subset112.tif', 'subset113.tif', 'subset114.tif', 'subset114.txt', 'subset115.tif', 'subset116.tif', 'subset117.tif', 'subset118.tif', 'subset119.tif', 'subset12.tif', 'subset120.tif', 'subset13.tif', 'subset13.txt', 'subset14.tif', 'subset15.tif', 'subset16.tif', 'subset17.tif', 'subset18.tif', 'subset19.tif', 'subset2.tif', 'subset20.tif', 'subset21.tif', 'subset22.tif', 'subset22.txt', 'subset23.tif', 'subset24.tif', 'subset25.tif', 'subset26.tif', 'subset27.tif', 'subset28.tif', 'subset29.tif', 'subset3.tif', 'subset30.tif', 'subset31.tif', 'subset32.tif', 'subset33.tif', 'subset34.tif', 'subset

2.2 Train pictures

In [38]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = train_folders,
                        destination_path_images = second_train + 'images/',
                        destination_path_labels = second_train + 'labels/',
                        destination_format=FORMAT_TO_COPY)

../data/8_final_dataset_second_version/train/labels/

S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC
['subset1.tif', 'subset10.tif', 'subset100.tif', 'subset100.txt', 'subset101.tif', 'subset101.txt', 'subset102.tif', 'subset102.txt', 'subset103.tif', 'subset104.tif', 'subset105.tif', 'subset106.tif', 'subset107.tif', 'subset108.tif', 'subset109.tif', 'subset11.tif', 'subset110.tif', 'subset111.tif', 'subset112.tif', 'subset112.txt', 'subset113.tif', 'subset114.tif', 'subset114.txt', 'subset115.tif', 'subset116.tif', 'subset117.tif', 'subset118.tif', 'subset119.tif', 'subset12.tif', 'subset120.tif', 'subset121.tif', 'subset122.tif', 'subset123.tif', 'subset124.tif', 'subset125.tif', 'subset126.tif', 'subset127.tif', 'subset128.tif', 'subset129.tif', 'subset13.tif', 'subset130.tif', 'subset131.tif', 'subset132.tif', 'subset14.tif', 'subset15.tif', 'subset16.tif', 'subset17.tif', 'subset18.tif', 'subset19.tif', 'subset2.tif', 'subset20.tif', 'subset21.tif', 'subse

2.3 Validation pictures

In [39]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = val_folders,
                        destination_path_images = second_val + 'images/',
                        destination_path_labels = second_val + 'labels/',
                        destination_format=FORMAT_TO_COPY)

../data/8_final_dataset_second_version/val/labels/

S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC
['subset1.tif', 'subset10.tif', 'subset100.tif', 'subset100.txt', 'subset101.tif', 'subset101.txt', 'subset102.tif', 'subset102.txt', 'subset103.tif', 'subset104.tif', 'subset105.tif', 'subset106.tif', 'subset107.tif', 'subset108.tif', 'subset109.tif', 'subset11.tif', 'subset110.tif', 'subset111.tif', 'subset112.tif', 'subset112.txt', 'subset113.tif', 'subset114.tif', 'subset114.txt', 'subset115.tif', 'subset116.tif', 'subset117.tif', 'subset118.tif', 'subset119.tif', 'subset12.tif', 'subset120.tif', 'subset121.tif', 'subset122.tif', 'subset123.tif', 'subset124.tif', 'subset125.tif', 'subset126.tif', 'subset127.tif', 'subset128.tif', 'subset129.tif', 'subset13.tif', 'subset130.tif', 'subset131.tif', 'subset132.tif', 'subset14.tif', 'subset15.tif', 'subset16.tif', 'subset17.tif', 'subset18.tif', 'subset19.tif', 'subset2.tif', 'subset20.tif', 'subset21.tif', 'subset2

# OLD

In [65]:
source_path = cutted_asf_data_directory
source_images_folders = test_folders
destination_path_images = final_dataset_images + 'test/'
destination_path_labels = final_dataset_labels + 'test/'
print(destination_path_labels)

for folder in source_images_folders:
    print()
    print(folder)
    # choose only files with txt annotations
    print(os.listdir(source_path + folder))
    print()
    labels=list()
    images=list()
    for file in os.listdir(source_path + folder):
        if file.endswith(".txt"):
            labels.append(file)
            image_file = file.replace(".txt", ".tif")
            images.append(image_file)
    # copy files - rename them while copying
    # labels
    for file in labels:
        src = source_path + '/' + folder + '/' + file
        dst = destination_path_labels + folder + file
        shutil.copy(src, dst)
    # pictures
    for file in images:
        src = source_path + '/' + folder + '/' + file
        dst = destination_path_images + folder + file
        shutil.copy(src, dst)
    

print(labels)
print()
print(images)

../data/7_final_dataset/labels/test/
S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC
['subset1.tif', 'subset1.txt', 'subset10.tif', 'subset10.txt', 'subset11.tif', 'subset12.tif', 'subset2.tif', 'subset3.tif', 'subset3.txt', 'subset4.tif', 'subset4.txt', 'subset5.tif', 'subset5.txt', 'subset6.tif', 'subset6.txt', 'subset7.tif', 'subset8.tif', 'subset9.tif', 'subset9.txt']

['subset1.txt', 'subset10.txt', 'subset3.txt', 'subset4.txt', 'subset5.txt', 'subset6.txt', 'subset9.txt']

['subset1.tif', 'subset10.tif', 'subset3.tif', 'subset4.tif', 'subset5.tif', 'subset6.tif', 'subset9.tif']
